In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn import metrics

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import scipy.stats as stats
from scipy.stats import mstats

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth',1000)

In [21]:
# CARGAMOS EL DATASET YA LIMPIO PARA TRABAJAR CON LOS MODELOS

df_LIMPIO=pd.read_csv('../data/df_LIMPIO_12_09.csv', index_col=0)
# df_LIMPIO

# REDUCIMOS LOS RANGOS ENTRE LOS QUE SE DIVIDIRÁ ENTRE 0 Y 4 PARA PROBAR CON DEEP LEARNING

for i in range(len(df_LIMPIO['rangos_precios'])):
    if df_LIMPIO['rangos_precios'].iloc[i] <= 50:
        df_LIMPIO['rangos_precios'].iloc[i] = 0

    elif 50 < df_LIMPIO['rangos_precios'].iloc[i] <= 100:
        df_LIMPIO['rangos_precios'].iloc[i] = 1

    elif 75 < df_LIMPIO['rangos_precios'].iloc[i] <= 150:
        df_LIMPIO['rangos_precios'].iloc[i] = 2

    else:
        df_LIMPIO['rangos_precios'].iloc[i] = 3

c:\Users\pieci\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [22]:
df_LIMPIO['rangos_precios'].value_counts()

0.0    7980
1.0    6090
3.0    2340
2.0    1910
Name: rangos_precios, dtype: int64

In [43]:
X=df_LIMPIO.drop(columns=[
                        ## VARIABLES TARGET
                        'rangos_precios',
                        'price',

                        'accommodates',
                        # 'beds',

                        'eur_distrito',
                        # 'elevator',
                        # 'latitude',
                        # 'longitude',
                        # 'amenities_len',
                        # 'balcony',
                        # 'pool',
                        # 'wifi',
                        'host_since_years',
                        'single_level',
                        'airport',
                        'workspace',
                        'breakfast',
                        'nespresso',
                        'dist_p_sol',
                        'pet',
                        'facturas_inc',
                        'reviews_per_month',
                        'review_scores_rating'
 ])

        
# y=df_LIMPIO['price_per_year']
# y=df_LIMPIO['price_per_month']
y=df_LIMPIO['rangos_precios']

features = list(X)

X_train_full, X_test, y_train_full, y_test = train_test_split(X,
                                                              y)

X_train, X_val, y_train, y_val = train_test_split(X_train_full,
                                                      y_train_full)

In [24]:
# y_train = y_train.astype("float32")/200
# y_test = y_test.astype("float32")/200

In [44]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10305, 16)
(4580, 16)
(10305,)
(4580,)


In [45]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [46]:
capas = [
    keras.layers.Flatten(input_shape= X_train.shape[1:]),
    keras.layers.Dense(units = 300, activation='relu'),
    keras.layers.Dense(units = 100, activation='relu'),
    keras.layers.Dense(units = 4, activation='softmax')
]

model = keras.models.Sequential(capas)

In [47]:
model.compile(
    optimizer = "sgd",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"])

In [48]:
history = model.fit(
    X_train,
    y_train,
    # batch_size = 128,
    epochs = 100,
    validation_data = (X_val, y_val) # validation_split = 0.1
)

Epoch 1/100
323/323 [==============================] - 1s 1ms/step - loss: 1.0282 - accuracy: 0.5913 - val_loss: 0.9398 - val_accuracy: 0.6294
Epoch 2/100
323/323 [==============================] - 0s 1ms/step - loss: 0.9270 - accuracy: 0.6340 - val_loss: 0.9101 - val_accuracy: 0.6358
Epoch 3/100
323/323 [==============================] - 0s 1ms/step - loss: 0.9066 - accuracy: 0.6399 - val_loss: 0.8984 - val_accuracy: 0.6384
Epoch 4/100
323/323 [==============================] - 0s 1ms/step - loss: 0.8968 - accuracy: 0.6419 - val_loss: 0.8928 - val_accuracy: 0.6399
Epoch 5/100
323/323 [==============================] - 0s 1ms/step - loss: 0.8899 - accuracy: 0.6437 - val_loss: 0.8931 - val_accuracy: 0.6355
Epoch 6/100
323/323 [==============================] - 0s 1ms/step - loss: 0.8849 - accuracy: 0.6439 - val_loss: 0.8826 - val_accuracy: 0.6457
Epoch 7/100
323/323 [==============================] - 0s 1ms/step - loss: 0.8802 - accuracy: 0.6448 - val_loss: 0.8827 - val_accuracy: 0.6425

In [49]:
pd.DataFrame(history.history).sort_values(by='val_accuracy', ascending=False).head(25)

,loss,accuracy,val_loss,val_accuracy
43,0.815101,0.672877,0.856080,0.657642
24,0.838199,0.659000,0.859811,0.657060
44,0.811194,0.674139,0.858850,0.656769
40,0.816490,0.670839,0.856630,0.655895
31,0.829119,0.665211,0.858516,0.655895
27,0.833417,0.660262,0.859272,0.655604
32,0.827085,0.662785,0.860417,0.655313
33,0.825212,0.664435,0.857581,0.655022
18,0.848278,0.658709,0.862778,0.655022
16,0.852309,0.655701,0.864896,0.654440


In [50]:
results = model.evaluate(X_test, y_test)
results

144/144 [==============================] - 0s 867us/step - loss: 0.9568 - accuracy: 0.6120


[0.9568202495574951, 0.6120087504386902]

In [51]:
predictions = model.predict(X_train).round(3)
print(predictions.shape)
predictions

323/323 [==============================] - 0s 637us/step
(10305, 4)


array([[0.835, 0.112, 0.003, 0.05 ],
       [0.831, 0.145, 0.009, 0.015],
       [0.081, 0.776, 0.096, 0.047],
       ...,
       [0.813, 0.068, 0.039, 0.08 ],
       [0.957, 0.019, 0.001, 0.023],
       [0.916, 0.029, 0.005, 0.05 ]], dtype=float32)

In [57]:
list_preds = []

for i in range(len(predictions)):
    # print(predictions_tot[i].argmax())
    list_preds.append(predictions[i].argmax())

df_preds = pd.DataFrame(y_train)
df_preds['preds'] = list_preds
df_preds.head(50)

,rangos_precios,preds
11481,3.0,0
9687,0.0,0
2764,0.0,1
2285,2.0,0
1030,1.0,0
10278,1.0,1
6533,0.0,0
18548,0.0,0
10107,0.0,0
15243,1.0,1


In [ ]:
holi.shape

In [ ]:
model.save("my_keras_model_12_09_20_03.h5")

In [ ]:
# Lo volvemos a cargar
model = keras.models.load_model("my_keras_model_12_09_20_03.h5")

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scaling the data
scaler = StandardScaler()

# Data scaled
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

# Modelo 1 - XGBClassifier
modelo = XGBClassifier(
    # max_depth=6, 
    # learning_rate=0.08
    )


le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

modelo.fit(X_train, y_train)

preds = modelo.predict(X_test)

print(modelo.score(X_train, y_train))
print(modelo.score(X_test, y_test))

print('Accuracy:', accuracy_score(y_test, preds))
print('Recall:', recall_score(y_test, preds, average='micro'))
print('Precision:', precision_score(y_test, preds, average='micro'))

0.8646288209606987
0.6839519650655022
Accuracy: 0.6839519650655022
Recall: 0.6839519650655022
Precision: 0.6839519650655022


In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

#XGBoost
params = {
    'max_depth': [10, 15, 20],
    'n_estimators': [50, 60, 70, 80],
    'learning_rate': [0.1]
}
xgrid = GridSearchCV(estimator=xgb.XGBClassifier(),           
                      param_grid=params, 
                      cv=10,
                      verbose=0) 
                      
xgrid.fit(X_train, y_train)

print(xgrid.best_score_)
print(xgrid.best_estimator_)
print(xgrid.best_params_)

0.6824964894875977
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=70, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, ...)
{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 70}


In [ ]:
holi.shape()

NameError: name 'holi' is not defined

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
# PROBAMOS CON UN MODELO BÁSICO DE REGRESIÓN LINEAL -> MALOS RESULTADOS EN TRAIN Y TEST

lm = LinearRegression()
lm.fit(X_train, y_train)

preds = lm.predict(X_test)
preds

print(modelo.score(X_train, y_train))
print(modelo.score(X_test, y_test))

# print('Accuracy:', accuracy_score(y_test, preds))
# print('Recall:', recall_score(y_test, preds, average='micro'))
# print('Precision:', precision_score(y_test, preds, average='micro'))

0.8646288209606987
0.6839519650655022


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [ ]:
# PROBAMOS CON UN MODELO DE REGRESIÓN POLINÓMICA DE GRADO 2 -> MALOS RESULTADOS EN TRAIN Y TEST

from sklearn.preprocessing import PolynomialFeatures

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

poly_reg = PolynomialFeatures(degree=2)
poly_reg.fit(X_train)
X_poly_train = poly_reg.transform(X_train)  

pol_reg_2 = LinearRegression()
pol_reg_2.fit(X_poly_train, y_train)

predictions = pol_reg_2.predict(poly_reg.transform(X_test))

print(pol_reg_2.score(X_train, y_train))
print(pol_reg_2.score(X_test, y_test))

print('Accuracy:', accuracy_score(y_test, predictions))
print('Recall:', recall_score(y_test, predictions, average='micro'))
print('Precision:', precision_score(y_test, predictions, average='micro'))

ValueError: X has 29 features, but LinearRegression is expecting 465 features as input.

In [ ]:
%pip install imblearn

In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier 

bbc = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)

#Train the classifier.
bbc.fit(X_train, y_train)
pred_y = bbc.predict(X_test)
mostrar_resultados(y_test, pred_y)
1
2
3
4
5
6
7
8
9
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)
 
#Train the classifier.
bbc.fit(X_train, y_train)
pred_y = bbc.predict(X_test)
mostrar_resultados(y_test, pred_y)

In [ ]:
# PROBAMOS CON UN MODELO DE REGRESIÓN POLINÓMICA DE GRADO 3 -> NO FUNCIONA!

from sklearn.preprocessing import PolynomialFeatures

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

poly_reg = PolynomialFeatures(degree=3)
poly_reg.fit(X_train)
X_poly_train = poly_reg.transform(X_train)  

pol_reg_2 = LinearRegression()
pol_reg_2.fit(X_poly_train, y_train)

predictions = pol_reg_2.predict(poly_reg.transform(X_test))

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print(pol_reg_2.score(X_poly_train, y_train))
print(pol_reg_2.score(poly_reg.transform(X_test), y_test))

MAE: 4.651611729444436
0.8240830101261405
0.7455660347996529


In [ ]:
# VAMOS A PROBAR CON UN ARBOL DE DECISION SENCILLO -> MUY BUENOS RESULTADOS EN TRAIN, MUY MALOS EN TEST

from sklearn.tree import DecisionTreeRegressor, plot_tree

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)

y_pred = dtr.predict(X_test)
y_pred

print('MAE', mean_absolute_error(y_test, y_pred))
print(dtr.score(X_train, y_train))
print(dtr.score(X_test, y_test))

MAE 0.5955240174672489
1.0
0.036680226738844524


In [ ]:
# plt.figure(figsize=(50,30))

# plot_tree(dtr, feature_names=X_train.columns, class_names='actual', fontsize=12);

In [ ]:
# PROBAMOS UN RANDOM FOREST. FUNCIONA BASTANTE BIEN Y MEJORAMOS MAE.

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

rnd_reg = RandomForestRegressor(n_estimators=450,
                                 random_state=42)
rnd_reg.fit(X_train, y_train)

print(rnd_reg.score(X_train, y_train))
print(rnd_reg.score(X_test, y_test))

predictions_rf = rnd_reg.predict(X_test)
# predictions_rf = np.exp(rnd_reg.predict(X_test))

print('MAE:', mean_absolute_error(y_test, predictions_rf))


0.9748445437570075
0.8142529142982222
MAE: 3.574236456063907


In [ ]:
# PROBAMOS ADABOOST. EMPEORA EN TODOS LOS ASPECTOS

from sklearn.ensemble import AdaBoostRegressor

ada_reg = AdaBoostRegressor(n_estimators=200,
                            random_state=42)
ada_reg.fit(X_train, y_train)

y_pred_ada_reg = ada_reg.predict(X_test)

print('MAE:', mean_absolute_error(y_test, y_pred_ada_reg))
print(ada_reg.score(X_train, y_train))
print(ada_reg.score(X_test, y_test))

MAE: 4.318972331180349
0.7660050342581729
0.7634373150293416


In [ ]:
# PROBAMOS GRADIENT BOOSTING. MALOS RESULTADOS

from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2,
                                 n_estimators=3,
                                 learning_rate=1.0,
                                 random_state=42)
gbrt.fit(X_train, y_train)


y_pred_gbrt = gbrt.predict(X_test)

print('MAE:', mean_absolute_error(y_test, y_pred_gbrt))
print(gbrt.score(X_train, y_train))
print(gbrt.score(X_test, y_test))

MAE: 4.3696823226529835
0.7690442854900079
0.7658233868175994


In [ ]:
# ESCALAMOS LOS DATOS PARA PROBAR DE NUEVO GRADIENT BOOSTING. LOS RESULTADOS NO MEJORAN EN NINGUN CASO.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scal = scaler.fit_transform(X_train)  # Valor mínimo 10 --> 0, Valor máximo 50 --> 1
X_test_scal = scaler.transform(X_test)

In [ ]:
gbrt = GradientBoostingRegressor(max_depth=2,
                                 n_estimators=3,
                                 learning_rate=1.0,
                                 random_state=42)
gbrt.fit(X_train_scal, y_train)


y_pred_gbrt = gbrt.predict(X_test_scal)

print('MAE:', mean_absolute_error(y_test, y_pred_gbrt))
print(gbrt.score(X_train_scal, y_train))
print(gbrt.score(X_test_scal, y_test))

MAE: 4.3696823226529835
0.7690442854900079
0.7658233868175994


In [ ]:
# CON XGBOOST TAMBIÉN TENEMOS MEJORES RESULTADOS. VAMOS A SEGUIR TRABAJANDO CON RANDOM FOREST Y XGBOOST PARA VER CUÁL LLEGA A RESULTADOS MEJORES.

import xgboost

xgb_reg = xgboost.XGBRegressor(random_state=42,
                            )

xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)

print('MAE:', mean_absolute_error(y_test, y_pred))
print(xgb_reg.score(X_train, y_train))
print(xgb_reg.score(X_test, y_test))

MAE: 3.969207647236344
0.9036892172753273
0.8048048392057872


In [ ]:
import xgboost

xgb_reg = xgboost.XGBRegressor(random_state=42
                               )

xgb_reg.fit(X_train_scal, y_train)

y_pred = xgb_reg.predict(X_test_scal)

print('MAE:', mean_absolute_error(y_test, y_pred))
print(xgb_reg.score(X_train_scal, y_train))
print(xgb_reg.score(X_test_scal, y_test))

MAE: 3.9666787078957153
0.9036892172753273
0.8049315526394905


In [ ]:
# VAMOS A HACER UNA PRUEBA CON REGRESION POL Y LASSOCV - NO ES ÚTIL

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV

# p = PolynomialFeatures()

# features_poly = p.fit_transform(X)

# poly_df = pd.DataFrame(features_poly, columns=p.get_feature_names_out())

# X_train, X_test, y_train, y_test = train_test_split(poly_df, y, random_state=28)

# ss = StandardScaler()
# ss.fit(X_train)
# X_train_sc = ss.transform(X_train)
# X_test_sc = ss.transform(X_test)

# y_train_log = np.log(y_train)
# y_test_log = np.log(y_test)

# print(X_train.shape)
# print(X_test.shape)

# lr = LinearRegression()
# lasso = LassoCV()


# lr.fit(X_train_sc, y_train_log)
# lasso.fit(X_train_sc, y_train_log)

# print(lr.score(X_train_sc, y_train_log))
# print(lr.score(X_test_sc, y_test_log))

# y_pred = np.exp(lasso.predict(X_test_sc))
# print('MAE:', mean_absolute_error(y_test, y_pred))